In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import string
from os import listdir
import os
import errno
import subprocess

import six # needed for Google Cloud client
# Imports the Google Cloud client library
# from google.cloud import language
# from google.cloud.language import enums
# from google.cloud.language import types

In [3]:
# !aws s3 ls 

In [4]:
!ls -lr recipes

total 0


In [5]:
recipe_dirname = 'recipes'
try:
    os.makedirs(recipe_dirname)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [8]:
datafiles = [
    'everyday-cooking.json',
    'fruits-and-vegetables.json',
    'pasta-and-noodles.json',
    'allRecipes_tony_categories.json',
    'main-dish.json',
    'meat-and-poultry.json',
    'lunch.json',
    'healthy-recipes.json',
    'holidays-and-events.json',
]

for filename in datafiles:
    if not os.path.isfile(os.path.join(recipe_dirname, filename)):
        print "Need to download: {}".format(os.path.join(recipe_dirname, filename))
        # TODO: use boto3 to download file
        subprocess.call(['aws', 's3', 'cp', 's3://RecipeVectors/'+filename, recipe_dirname])
    else:
        print "datafile already present: {}".format(os.path.join(recipe_dirname, filename))

Need to download: recipes/everyday-cooking.json
datafile already present: recipes/fruits-and-vegetables.json
Need to download: recipes/pasta-and-noodles.json
Need to download: recipes/allRecipes_tony_categories.json
Need to download: recipes/main-dish.json
Need to download: recipes/meat-and-poultry.json
datafile already present: recipes/lunch.json
Need to download: recipes/healthy-recipes.json
Need to download: recipes/holidays-and-events.json


In [21]:
df =pd.DataFrame()
cnt=0
for file in listdir('recipes'):
#    print 'recipes/'+file
    df = df.append(pd.read_json('recipes/'+file, dtype={}))
    print 'Loaded {0} recipes from {1}'.format(len(df)-cnt,file)
    cnt=len(df)
print 'Total {0}'.format(len(df))
df.sample(5)

Loaded 29453 recipes from everyday-cooking.json
Loaded 24107 recipes from fruits-and-vegetables.json
Loaded 3969 recipes from pasta-and-noodles.json
Loaded 28877 recipes from allRecipes_tony_categories.json
Loaded 14456 recipes from main-dish.json
Loaded 14729 recipes from meat-and-poultry.json
Loaded 880 recipes from lunch.json
Loaded 4235 recipes from healthy-recipes.json
Loaded 12679 recipes from holidays-and-events.json
Total 133385


,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url
11146,"[Drop Cookies, Cookies, Desserts, Everyday Coo...",PT30M,NaN,"[2 persimmons, 1/2 cup shortening, 1 cup white...",[Peel and de-stem the persimmons and process t...,Moist Persimmon Cookie,4.41,16,http://allrecipes.com/recipe/19268/moist-persi...
962,"[Wheat Bread, Whole Grain Bread, Yeast Bread, ...",NaN,NaN,"[1/2 cup warm water, 3 (.25 ounce) packages ac...","[In the mixing bowl of an electric mixer, stir...",Fabulous Homemade Bread,4.75,508,http://allrecipes.com/recipe/6700/fabulous-hom...
19543,"[Wheat Bread, Whole Grain Bread, Yeast Bread, ...",PT3H10M,NaN,[1 1/2 cups warm water (100 to 110 degrees F/4...,"[Pour the warm water into a large bowl, and wh...",Fabulous Homemade Bread For the Food Processor,4.63,24,http://allrecipes.com/recipe/219073/fabulous-h...
2059,"[Flat Iron Steak, Beef Steaks, Everyday Cookin...",PT2H28M,NaN,"[1 (2 pound) flat iron steak, 2 1/2 tablespoon...",[Place the steak inside of a large resealable ...,Perfect Flat Iron Steak,4.22,108,http://allrecipes.com/recipe/150685/perfect-fl...
16596,"[Potato Side Dishes, Curry Side Dishes, Side D...",PT30M,NaN,"[1 cup vegetable oil for frying, or as needed,...",[Pour enough oil into a frying pan to cover th...,Indian Curry Potato Fries,5.00,6,http://allrecipes.com/recipe/222808/indian-cur...


In [38]:
from collections import defaultdict
import operator

categories = defaultdict(int)
for idx, row in df.iterrows():
    # print type(row['categories'])
    if  type(row['categories']) != float:
        for cat in row['categories']:
            categories[cat] += 1 
print 'Top 30 Categories:'
for item in sorted(categories.items(), reverse=True, key=operator.itemgetter(1))[:30]:
    print item

Top 30 Categories:
(u'Recipes', 130426)
(u'Everyday Cooking', 101321)
(u'Main Dishes', 48355)
(u'Desserts', 26288)
(u'Appetizers and Snacks', 12636)
(u'Pasta by Shape', 10978)
(u'Chicken Breasts', 9792)
(u'Side Dishes', 9275)
(u'Soups, Stews and Chili', 8361)
(u'Cookies', 7791)
(u'Fruit Desserts', 7737)
(u'Pasta Main Dishes', 7491)
(u'Salad Recipes', 7424)
(u'Bread Recipes', 7294)
(u'Cake Recipes', 7006)
(u'Breakfast and Brunch', 6862)
(u'Drinks', 6822)
(u'Vanilla Extract', 6737)
(u'Italian Recipes', 6717)
(u'Faceless Recipes', 6703)
(u'Asian Recipes', 6697)
(u'Trusted Brands', 6075)
(u'Mexican Recipes', 5537)
(u'Soup', 5301)
(u'Chocolate Desserts', 5197)
(u'Dips and Spreads', 4903)
(u'Baked and Roasted Chicken', 4482)
(u'Vegan Recipes', 4478)
(u'Cheese Appetizers', 4349)
(u'Fish Recipes', 4319)


In [41]:
import re

def convertCookingTimeToMinutes(text):
    if type(text) == float:
        return None
    text = text.replace('PT','')
    arr = re.findall(r'[0-9]+', text)
    minute = 0
    if len(arr) == 2:
        minute = int(arr[0])* 60 + int(arr[1])
    if len(arr) == 1:
        if 'M' in text:
            minute = int(arr[0])
        if 'H' in text:
            minute = int(arr[0]) * 60
    return minute
df['cookingTimeMinutes'] = df['cookingTime'].apply(convertCookingTimeToMinutes)
df[['cookingTime','cookingTimeMinutes']].sample(5)

,cookingTime,cookingTimeMinutes
6230,NaN,NaN
16844,PT50M,50.0
9283,PT1H10M,70.0
27428,PT1H20M,80.0
4249,PT2H,120.0


In [42]:
pd.set_option('display.max_colwidth', -1)
df.iloc[[10]]['ingredients']

10    [2 cups all-purpose flour, 1/2 teaspoon baking soda, 1/2 teaspoon salt, 3/4 cup unsalted butter, melted, 1 cup packed brown sugar, 1/2 cup white sugar, 1 tablespoon vanilla extract, 1 egg, 1 egg yolk, 2 cups semisweet chocolate chips, Add all ingredients to list, Add all ingredients to list]
Name: ingredients, dtype: object

In [43]:
df[['ingredients']].values[10][0]

[u'2 cups all-purpose flour',
 u'1/2 teaspoon baking soda',
 u'1/2 teaspoon salt',
 u'3/4 cup unsalted butter, melted',
 u'1 cup packed brown sugar',
 u'1/2 cup white sugar',
 u'1 tablespoon vanilla extract',
 u'1 egg',
 u'1 egg yolk',
 u'2 cups semisweet chocolate chips',
 u'Add all ingredients to list',
 u'Add all ingredients to list']

In [44]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        translate_table = dict((ord(char), unicode(translate_to))
                               for char in not_letters)
    else:
        assert isinstance(to_translate, str)
        translate_table = string.maketrans(not_letters,
                                           translate_to
                                              *len(not_letters))
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        ###
        'loaf', 'loaves',
        'pouch', 'pouches',
        'wedge', 'wedges',
        'drop', 'drops',
        'amount', 'amounts',
        'bulk', 'bulks',
        'roll', 'rolls',
        'hint', 'hints',
        'round', 'rounds',
        'cube', 'cubes',
        'husk', 'husks',
        'envelope', 'envelopes',
        'container', 'containers',
        'dash', 'dashes',
        'bitesize', 'bitesized',
        'bite', 'sized',
        'size',
        'taste', 'desired',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'tub', 'tubs',
        'slab', 'slabs',
        'sprig', 'sprigs',
        'stalk', 'stalks',
        'matchstick', 'matchsticks',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
        'bag', 'bags',
        'dash', 'dashes',
        'container', 'containers',
        'envelope', 'envelopes',
        'sheet', 'sheets',
        'whole',
        'extra', 'extras',
        'dozen', 'dozens',
        'half', 'halves',
        'third', 'thirds',
        'quarter', 'quarters',
        'eighth', 'eighths',
    ]
    
    preparatory_descriptions = [
        'baked',
        'beat', 'beaten',
        'blanched',
        'blended',
        'boiled',
        'broiled',
        'broken',
        'browned',
        'canned',
        'chilled',
        'chipped',
        'chopped',
        'cleaned',
        'cooked',
        'cooled',
        'cored',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'defrosted',
        'diced',
        'divided',
        'drained',
        'dried',
        'filleted',
        'filtered',
        'flaked',
        'fresh',
        'fried',
        'frozen',
        'grated',
        'grilled',
        'ground',
        'halved',
        'hardened',
        'heated',
        'hulled',
        'juiced',
        'julienned',
        'marinated',
        'mashed',
        'melted',
        'microwaved',
        'minced',
        'mixed',
        'opened',
        'packed',
        'peeled',
        'pitted',
        'prepared',
        'pressed',
        'quartered',
        'raw',
        'refrigerated',
        'removed',
        'rinsed',
        'ripe',
        'ripened',
        'ripped',
        'roasted',
        'seasoned',
        'sectioned',
        'seeded',
        'separated',
        'shaved',
        'shredded',
        'shucked',
        'sifted',
        'smashed',
        'smoked',
        'sliced',
        'slivered',
        'softened',
        'split',
        'squeezed',
        'steamed',
        'stemmed',
        'soaked',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'unopened',
        'unpeeled',
        'warmed',
        'washed',
        'zested',
    ]
    
    other_words_to_filter = [
        'a', 'an',
        'about',
        'add',
        'all',
        'and',
        'as',
        'at',
        'in',
        'into',
        'ingredients',
        'for',
        'less',
        'list',
        'more',
        'needed',
        'of',
        'optional',
        'or',
        'other',
        'such',
        'the',
        'to',
        'very',
        'with',
        'thin', 'thinly',
        'thick', 'thickly',
        'fine', 'finely',
        'coarse', 'coarsely',
        'rough', 'roughly',
        'approximately',
        'freshly',
        'large', 'extralarge', 'medium', 'small', 'extrasmall',
        'hard', 'firm', 'soft',
        'lengthwise', 'widthwise', 'piecewise', 'crosswise', 'sideways',
        'horizontally', 'diagonally', 'vertically',
    ]
    
    #print "Before..."
    #print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    #print cleaned_ingred_list
    
    # split on whitespace
    # throw out meas units
    #print
    #print "After..."
    ingreds_only = [' '.join(w for w in ingred_item.split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
         for ingred_item in cleaned_ingred_list
         ]
    ingreds_only = filter(None, ingreds_only) # remove empty strings at end
    #print ingreds_only
    
    return ingreds_only

extract_ingredients(df[['ingredients']].values[867][0])

[u'turkey',
 u'dry onion soup mix',
 u'black pepper',
 u'garlic powder',
 u'soy sauce',
 u'egg lightly',
 u'hamburger buns']

In [45]:
print len(df[['ingredients']].values)
print df[['ingredients']].values.shape
print type(df[['ingredients']].values)
#print df[['ingredients']].values
print 

print len(df[['ingredients']].values[867])
print df[['ingredients']].values[867].shape
print type(df[['ingredients']].values[867])
#print df[['ingredients']].values[867]
print

#df[['ingredients']].values[][0]

print len(df[['ingredients']].values[867][0])
#print df[['ingredients']].values[867][0].shape
print type(df[['ingredients']].values[867][0])
#print df[['ingredients']].values[867][0]
print

print len(np.squeeze(df[['ingredients']].values))
print np.squeeze(df[['ingredients']].values).shape
print type(np.squeeze(df[['ingredients']].values))
print np.squeeze(df[['ingredients']].values)
print 

print len(np.squeeze(df[['ingredients']].values)[867])
#print np.squeeze(df[['ingredients']].values).shape
print type(np.squeeze(df[['ingredients']].values)[867])
print np.squeeze(df[['ingredients']].values)[867]
print 

133385
(133385, 1)
<type 'numpy.ndarray'>

1
(1,)
<type 'numpy.ndarray'>

9
<type 'list'>

133385
(133385,)
<type 'numpy.ndarray'>
[ [u'1/2 cup canola oil', u'1/3 cup tarragon vinegar', u'1 tablespoon white sugar', u'1 teaspoon chopped fresh thyme', u'1/2 teaspoon dry mustard', u'2 cloves garlic, minced', u'1 (8 ounce) can artichoke hearts, drained and quartered', u'5 cups romaine lettuce - rinsed, dried, and chopped', u'1 red bell pepper, cut into strips', u'1 carrot, grated', u'1 red onion, thinly sliced', u'1/4 cup black olives', u'1/4 cup pitted green olives', u'1/2 cucumber, sliced', u'2 tablespoons grated Romano cheese', u'ground black pepper to taste', u'Add all ingredients to list', u'Add all ingredients to list']
 [u'3 cups all-purpose flour', u'1 teaspoon salt', u'1 teaspoon baking soda', u'1 teaspoon baking powder', u'1 tablespoon ground cinnamon', u'3 eggs', u'1 cup vegetable oil', u'2 1/4 cups white sugar', u'3 teaspoons vanilla extract', u'2 cups grated zucchini', u'1 cup

In [46]:
ingreds_arr_of_lists = np.squeeze(df[['ingredients']].values)
vect_extract_ingred = np.vectorize(extract_ingredients, otypes=[list])

processed_ingreds_arr_of_lists = vect_extract_ingred(ingreds_arr_of_lists)

print type(processed_ingreds_arr_of_lists)
print len(processed_ingreds_arr_of_lists)
print processed_ingreds_arr_of_lists.size



<type 'numpy.ndarray'>
133385
133385


In [47]:
print processed_ingreds_arr_of_lists
#unique_vocab = set(np.hstack(processed_ingreds_arr_of_lists))

[ [u'canola oil', u'tarragon vinegar', u'white sugar', u'thyme', u'dry mustard', u'garlic', u'artichoke hearts', u'romaine lettuce', u'red bell pepper', u'carrot', u'red onion', u'black olives', u'green olives', u'cucumber', u'romano cheese', u'black pepper']
 [u'allpurpose flour', u'salt', u'baking soda', u'baking powder', u'cinnamon', u'eggs', u'vegetable oil', u'white sugar', u'vanilla extract', u'zucchini', u'walnuts']
 [u'brown sugar', u'ketchup', u'lean beef', u'milk', u'eggs', u'salt', u'black pepper', u'onion', u'ginger', u'saltine cracker crumbs']
 ...,
 [u'beef', u'worcestershire sauce', u'garlic salt', u'black pepper', u'american cheese kraft\xae', u'hamburger buns']
 [u'strawberries', u'white sugar', u'cold water', u'lime', u'mint']
 [u'matzo', u'tomato sauce', u'salt pepper', u'swiss cheese', u'mozzarella cheese', u'parmesan cheese', u'oregano']]


In [49]:
flattened_ingred_list = np.hstack(processed_ingreds_arr_of_lists)
print len(flattened_ingred_list)
print flattened_ingred_list

unique_vocab = sorted(list(set(flattened_ingred_list)))
print len(unique_vocab)
print unique_vocab

1248465
[u'canola oil' u'tarragon vinegar' u'white sugar' ..., u'mozzarella cheese'
 u'parmesan cheese' u'oregano']
25360
[u'abalone without shell', u'absinthe', u'acai berry pulp', u'acai berry pulp unsweetened', u'acai berry sorbet', u'acai juice', u'acai powder', u'acai pulp sambazon \xae', u'accompaniments', u'achar masala', u'achiote annatto seeds', u'achiote paste', u'achiote powder', u'achiote seed', u'achiote seeds', u'acini de pepe pasta', u'acini di pepe pasta', u'acorn squash', u'acorn squashes', u'act ii\xae fat free popcorn', u'active dry yeast', u'active dry yeast bread machine yeast', u'active dry yeast fleischmanns activedry yeast\xae', u'active dry yeast fleischmanns rapidrise\xae', u'addins', u'additional butter margarine', u'additional butter sugar garnish', u'additional candy sprinkles', u'additional cheddar cheese', u'additional cocktail sauce dipping', u'additional extravirgin olive oil', u'additional extravirgin olive oil brushing', u'additional extravirgin olive

In [50]:
flat_ingred_arr = [' '.join(lst) for lst in ingreds_arr_of_lists]

cv = CountVectorizer(vocabulary=unique_vocab)
recipe_ingredient_matrix = cv.fit_transform(flat_ingred_arr)

print type(recipe_ingredient_matrix)
print recipe_ingredient_matrix.size

print recipe_ingredient_matrix

<class 'scipy.sparse.csr.csr_matrix'>
1405320
  (0, 557)	1
  (0, 3617)	1
  (0, 3990)	1
  (0, 6329)	1
  (0, 6962)	1
  (0, 9634)	1
  (0, 13355)	1
  (0, 15270)	1
  (0, 15621)	1
  (0, 15739)	2
  (0, 15747)	1
  (0, 16803)	2
  (0, 19513)	1
  (0, 22011)	1
  (0, 22660)	1
  (0, 22889)	1
  (0, 24174)	1
  (0, 24522)	1
  (1, 983)	2
  (1, 5025)	1
  (1, 7513)	1
  (1, 8239)	1
  (1, 15621)	1
  (1, 19834)	1
  (1, 22011)	1
  :	:
  (133381, 23768)	2
  (133381, 24230)	1
  (133381, 24522)	2
  (133382, 1398)	1
  (133382, 3990)	1
  (133382, 9634)	1
  (133382, 10890)	1
  (133382, 16803)	1
  (133382, 19834)	1
  (133382, 20032)	1
  (133383, 13512)	1
  (133383, 14968)	1
  (133383, 21846)	1
  (133383, 22011)	1
  (133383, 24293)	1
  (133383, 24522)	1
  (133384, 3990)	3
  (133384, 14404)	1
  (133384, 15136)	1
  (133384, 15966)	1
  (133384, 16373)	1
  (133384, 16803)	1
  (133384, 19834)	1
  (133384, 20032)	1
  (133384, 22977)	1


In [51]:
svd = TruncatedSVD(n_components=50)
reduced_recipe_ingredient_matrix = svd.fit_transform(recipe_ingredient_matrix)

print type(reduced_recipe_ingredient_matrix)
print reduced_recipe_ingredient_matrix.size

<type 'numpy.ndarray'>
6669250


In [55]:
df['ingredients'] = df['ingredients'].apply(extract_ingredients)
df.sample(5)

,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url,cookingTimeMinutes
1060,"[Meatball Appetizers, Beef Appetizers, Meat and Poultry Appetizers, Appetizers and Snacks, Everyday Cooking, Recipes, Winter Appetizers, Slow Cooker Beef Main Dishes, Slow Cooker Appetizers, Christmas Appetizers]",PT1H50M,These meatballs are slow-cooked in a sweet and sour sauce.,"[fluid chile sauce, lemon juice, grape jelly, lean beef, egg, onion, salt]","[Whisk together the chili sauce, lemon juice and grape jelly. Pour into slow cooker and simmer over low heat until warm., Combine ground beef, egg, onion and salt. Mix well and form into 1 inch balls. Add to sauce and simmer for 1 1/2 hours.]",Sweet and Sour Meatballs,4.24,212,http://allrecipes.com/recipe/15551/sweet-and-sour-meatballs/,110.0
7723,"[Hamburgers, Burgers, Main Dishes, Recipes, Low-Sodium Beef Main Dishes, Cumin, Low-Carb Beef Main Dishes, Summer Main Dishes, BBQ & Grilled Beef Burgers, Corn Recipes]",NaN,NaN,"[beef, cumin, cumin seeds, corn kernels, salt pepper]","[Preheat grill for high heat., In a large bowl, combine beef, ground cumin, cumin seeds, corn, and salt and pepper. Mix together and form into patties., Lightly oil grate, and place patties onto grill. Cook for 3 to 4 minutes per side, or until cooked through.]",Cumin Corn Burgers,3.46,10,http://allrecipes.com/recipe/14575/cumin-corn-burgers/,NaN
11883,"[Rice Side Dishes, Side Dishes, Brazilian Recipes, Everyday Cooking, Recipes, Sugar-Free Side Dishes, Olympics Recipes, Dairy-Free Side Dishes, South American Recipes, Egg-Free Side Dishes]",PT45M,NaN,"[longgrain white rice, onion, garlic, vegetable oil, salt, hot water]","[Place the rice in a colander and rinse thoroughly with cold water; set aside., Heat the oil in a saucepan over medium heat. Cook the onion in the oil for one minute. Stir in the garlic and cook until the garlic is golden brown. Add the rice and salt and cook and stir until the rice begins to brown. Pour hot water over rice mixture and stir. Reduce heat to low, cover the saucepan, and allow to simmer until the water has been absorbed, 20 to 25 minutes.]",Brazilian White Rice,4.42,301,http://allrecipes.com/recipe/144308/brazilian-white-rice/,45.0
1263,"[Beef Soup, Soup, Soups, Stews and Chili, Italian Recipes, Pasta by Shape, Everyday Cooking, Recipes, MyPlate Soups and Stews, Italian Soups and Stews, Basil]",PT25M,NaN,"[extralean beef, egg lightly, italianseasoned breadcrumbs, parmesan cheese, basil leaves, italian flat leaf parsley, green onions, chicken broth, escarole spinach may be substituted, lemon, orzo riceshaped pasta, parmesan cheese topping]","[Mix together the meat, egg, bread crumbs, cheese, basil, parsley, and green onions; shape into 3/4 inch balls., Pour broth into a large saucepan over high heat. When boiling, drop in meatballs. Stir in escarole, lemon zest and orzo. Return to a boil; reduce heat to medium. Cook at a slow boil for 10 minutes or until orzo is tender, stirring frequently. Serve sprinkled with cheese.]",California Italian Wedding Soup,4.66,171,http://allrecipes.com/recipe/35781/california-italian-wedding-soup/,25.0
4399,"[Fried Rice, Asian Recipes, Everyday Cooking, Main Dishes, Recipes, Allrecipes Allstars Main Dishes, Snow Pea Recipes, Rice Leftovers, Dairy-Free Main Dishes, 200 Calorie Main Dishes]",PT12H30M,NaN,"[instant rice, chicken bouillon, snow peas, onions, bean sprouts, eggs, vegetable oil, soy sauce]","[In a saucepan bring 4 cups of water to a boil. Add chicken bouillon and rice, and stir. Cover, remove from heat and let stand 5 minutes, or until liquid is absorbed. Refrigerate overnight., To the rice add snow peas, onions and bean sprouts., In a small skillet over medium heat, scramble the eggs; add to rice mixture., Heat oil in a large skillet or wok over medium heat. Fry the rice mixture with soy sauce until liquid evaporates; be careful not to fry until crisp.]",Fried Rice II,4.17,100,http://allrecipes.com/recipe/23103/f

In [62]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

def makePipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', KMeans(n_clusters=2, init='k-means++', max_iter=100, n_init=1))
    ])
    return pipeline

def trainModel(X, Y):
    model = makePipeline()
    model.fit(X, Y)
    return model

X = [' '.join(f) for f in df['ingredients'].values]
Y = df['categories'].values   # Roy: I think this needs to change to a 

model = trainModel(X, Y)

In [63]:
true_k = len(np.unique(Y))
true_k

/home/roy_gvirtsman/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


52632

In [66]:
preds = model.predict(X)

In [65]:
uniq = np.unique(Y)
pred_labeled = [uniq[1] if p == 0 else uniq[0] for p in preds]
print confusion_matrix(pred_labeled, Y)

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.

In [73]:
from sklearn.metrics import classification_report
print classification_report(pred_labeled, Y)

             precision    recall  f1-score   support

   bbqbread       0.57      0.92      0.70      3253
     salads       0.97      0.78      0.86     10282

avg / total       0.87      0.81      0.82     13535



In [59]:
print("Top terms per cluster:")

vectorizer = model.named_steps['vect']
km = model.named_steps['clf']
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = vectorizer.get_feature_names()
for i in range(true_k):
    print "Cluster %d:" % i
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind]
    print


Top terms per cluster:
Cluster 0:
 pepper
 cheese
 garlic
 salt
 black
 oil
 onion
 red
 sauce
 green

Cluster 1:
 flour
 baking
 sugar
 allpurpose
 white
 milk
 butter
 powder
 salt
 eggs



In [5]:
def syntax_text(text):
    """Detects syntax in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    for token in tokens:
        print(u'{}: {}'.format(pos_tag[token.part_of_speech.tag],
                               token.text.content))

In [7]:
syntax_text('three cored and peeled green tomatoes')

NUM: three
VERB: cored
CONJ: and
VERB: peeled
ADJ: green
NOUN: tomatoes


In [ ]:
ingred_list = [u'3 avocados - peeled, pitted, and mashed',
 u'1 lime, juiced',
 u'1 teaspoon salt',
 u'1/2 cup diced onion',
 u'3 tablespoons chopped fresh cilantro',
 u'2 roma (plum) tomatoes, diced',
 u'1 teaspoon minced garlic',
 u'1 pinch ground cayenne pepper (optional)',
 u'Add all ingredients to list',
 u'Add all ingredients to list']

for ingre

In [2]:
!ls recipeVectors

allRecipes_recipes.json
allRecipes_recipes_appetizers_salads.json
recipeVectors
recipes
scrapy.cfg
test.py


In [11]:
# RNN implementation

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import sys
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
def getMultiClassData():
    filename2 = './recipeVectors/allRecipes_recipes.json'
#     filename2 = '../../data/cleandata/sunbasket_noapp.csv'
    df = pd.read_json(filename2)
    return df

def concatIngredients(arr):
    return ','.join(arr).encode('ascii', 'ignore')


def getTopCategory(arr):
    if arr == arr and len(arr) > 0:
        return arr[0].encode('ascii', 'ignore')

    return None


def processKerasModel(df_):
    embedding_length = 100
    top_words = 10000
    df = df_.copy()
    df['features'] = df['ingredients'].apply(concatIngredients)
    df['label'] = df['categories'].apply(getTopCategory)
    df = df[df['label'].astype(str) != 'nan']
    le = preprocessing.LabelEncoder()
    features = df['features'].values
    labels = df['label'].values
    
    le.fit(np.unique(labels))
    print list(le.classes_), 'num of labels:',len(np.unique(labels))
    labels = le.transform(labels) 
    print labels[:10]
#     X_train, Y_train, X_test, Y_test = ut.simpleSplit(features, labels)

    tokenizer = Tokenizer(nb_words=top_words)
    tokenizer.fit_on_texts(features)
    sequences = tokenizer.texts_to_sequences(features)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=embedding_length)

#     labels = np_utils.to_categorical(np.asarray(labels))
    print('Shape of data tensor:', data.shape)
    print('Shape of label tensor:', labels.shape)

    # split the data into a training set and a validation set
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices]
    labels = labels[indices]
    nb_validation_samples = int(0.2 * data.shape[0])

    X_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    X_test = data[-nb_validation_samples:]
    y_test = labels[-nb_validation_samples:]
    
    embedding_vecor_length = 100
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=embedding_length))
#     model.add(Dropout(0.5))
    model.add(LSTM(100)) #, return_sequences=True))
    model.add(Dropout(0.4))
#     model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(len(np.unique(labels)), activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy',
#         loss='sparse_categorical_crossentropy',
#               optimizer='rmsprop',
                optimizer='adam',
#               loss='sparse_categorical_crossentropy',
                metrics=['acc'])
#               metrics=['accuracy'])
#     model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
    print(model.summary())
    
    model.fit(X_train, y_train, epochs=10, batch_size=100)
    # Final evaluation of the model
    return model, X_test, y_test
finalset = getMultiClassData()

print finalset.sample(5)

dl_model, X_test, y_test = processKerasModel(finalset)
scores = dl_model.evaluate(X_test, y_test, verbose=1)  
print scores




                                             categories cookingTime  \
2438  [Stew, Soups, Stews and Chili, Indian Recipes,...        PT1H   
8321  [Vegetable Side Dishes, Side Dishes, Everyday ...         NaN   
3401  [The Sorghum Checkoff, Bean and Pea Soups, Sou...     PT1H10M   
1749  [Apple Salad, Fruit Salads, Salad Recipes, Eve...       PT15M   
3901  [Squash Soup, Vegetable Soup, Curry Soup, Soup...      PT2H3M   

     description                                        ingredients  \
2438         NaN  [1 eggplant, 1 teaspoon vegetable oil, 2 table...   
8321         NaN  [2 heads   cauliflower, 1 1/2 cups sour cream,...   
3401         NaN  [1/2 cup whole grain sorghum, uncooked, 3 cups...   
1749         NaN  [1 cantaloupe - peeled, seeded, and cubed, 2 r...   
3901         NaN  [1 butternut squash, halved and seeded, 1 tabl...   

                                       instructionSteps  \
2438  [Preheat an oven to 200 degrees F (95 degrees ...   
8321  [Preheat oven to 350 d

NameError: global name 'ut' is not defined